# New dim_shapes

In [ ]:
import datetime
import geopandas as gpd
import os
import pandas as pd

os.environ["CALITP_BQ_MAX_BYTES"] = str(130_000_000_000)

from calitp.tables import tbl
from calitp import query_sql
from siuba import *

#import shared_utils

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


## With string as date

In [ ]:
SELECTED_DATE = "2022-1-1"

start = datetime.datetime.now()

df = (tbl.views.gtfs_schedule_dim_shapes()
      >> filter(_.calitp_extracted_at <= SELECTED_DATE, 
                _.calitp_deleted_at > SELECTED_DATE
               )
      >> filter((_.calitp_itp_id==17) | (_.calitp_itp_id==4))
      >> select(_.calitp_itp_id, _.calitp_url_number, _.shape_id)
      >> inner_join(_, 
                    tbl.views.gtfs_schedule_dim_shapes_geo(),
                    ["calitp_itp_id", "calitp_url_number", "shape_id"])
      #>> show_query()
     )
end = datetime.datetime.now()
print(end - start)

0:00:00.392964


/opt/conda/lib/python3.9/site-packages/pybigquery/sqlalchemy_bigquery.py:879: SAWarning: Did not recognize type 'GEOGRAPHY' of column 'pt_array'
  util.warn(


In [ ]:
query = ('''
SELECT `anon_1`.`calitp_itp_id`, `anon_1`.`calitp_url_number`, `anon_1`.`shape_id`, `anon_2`.`calitp_extracted_at`, `anon_2`.`pt_array`, `anon_2`.`calitp_deleted_at` 
FROM (SELECT `anon_3`.`calitp_itp_id` AS `calitp_itp_id`, `anon_3`.`calitp_url_number` AS `calitp_url_number`, `anon_3`.`shape_id` AS `shape_id` 
FROM (SELECT `anon_4`.`calitp_itp_id` AS `calitp_itp_id`, `anon_4`.`calitp_url_number` AS `calitp_url_number`, `anon_4`.`shape_id` AS `shape_id`, `anon_4`.`shape_pt_lat` AS `shape_pt_lat`, `anon_4`.`shape_pt_lon` AS `shape_pt_lon`, `anon_4`.`shape_pt_sequence` AS `shape_pt_sequence`, `anon_4`.`shape_dist_traveled` AS `shape_dist_traveled`, `anon_4`.`calitp_extracted_at` AS `calitp_extracted_at`, `anon_4`.`calitp_hash` AS `calitp_hash`, `anon_4`.`shape_key` AS `shape_key`, `anon_4`.`calitp_deleted_at` AS `calitp_deleted_at` 
FROM (SELECT `views.gtfs_schedule_dim_shapes`.`calitp_itp_id` AS `calitp_itp_id`, `views.gtfs_schedule_dim_shapes`.`calitp_url_number` AS `calitp_url_number`, `views.gtfs_schedule_dim_shapes`.`shape_id` AS `shape_id`, `views.gtfs_schedule_dim_shapes`.`shape_pt_lat` AS `shape_pt_lat`, `views.gtfs_schedule_dim_shapes`.`shape_pt_lon` AS `shape_pt_lon`, `views.gtfs_schedule_dim_shapes`.`shape_pt_sequence` AS `shape_pt_sequence`, `views.gtfs_schedule_dim_shapes`.`shape_dist_traveled` AS `shape_dist_traveled`, `views.gtfs_schedule_dim_shapes`.`calitp_extracted_at` AS `calitp_extracted_at`, `views.gtfs_schedule_dim_shapes`.`calitp_hash` AS `calitp_hash`, `views.gtfs_schedule_dim_shapes`.`shape_key` AS `shape_key`, `views.gtfs_schedule_dim_shapes`.`calitp_deleted_at` AS `calitp_deleted_at` 
FROM `views.gtfs_schedule_dim_shapes`) AS `anon_4` 
WHERE `anon_4`.`calitp_extracted_at` <= '2022-1-1' AND `anon_4`.`calitp_deleted_at` > '2022-1-1') AS `anon_3` 
WHERE `anon_3`.`calitp_itp_id` IN (4, 17) AS `anon_1` JOIN (SELECT `views.gtfs_schedule_dim_shapes_geo`.`calitp_itp_id` AS `calitp_itp_id`, `views.gtfs_schedule_dim_shapes_geo`.`calitp_url_number` AS `calitp_url_number`, `views.gtfs_schedule_dim_shapes_geo`.`shape_id` AS `shape_id`, `views.gtfs_schedule_dim_shapes_geo`.`calitp_extracted_at` AS `calitp_extracted_at`, `views.gtfs_schedule_dim_shapes_geo`.`calitp_deleted_at` AS `calitp_deleted_at`, `views.gtfs_schedule_dim_shapes_geo`.`pt_array` AS `pt_array` 
FROM `views.gtfs_schedule_dim_shapes_geo`) AS `anon_2` ON `anon_1`.`calitp_itp_id` = `anon_2`.`calitp_itp_id` AND `anon_1`.`calitp_url_number` = `anon_2`.`calitp_url_number` AND `anon_1`.`shape_id` = `anon_2`.`shape_id`
'''
)

In [ ]:
"""
INCLUDE_ITP_LIST = [17, 4]

def unpack_list_make_or_statement(INCLUDE_ITP_LIST):
    new_cond = ""

    for i in range(0, len(INCLUDE_ITP_LIST)):
        cond = f"`anon_3`.`calitp_itp_id` = {INCLUDE_ITP_LIST[i]}"
        if i==0:
            new_cond = cond
        else:
            new_cond = new_cond + " OR " + cond
    return new_cond


or_condition = unpack_list_make_or_statement(INCLUDE_ITP_LIST)

query = (
    f'''
SELECT 
  `anon_1`.`calitp_itp_id`, 
  `anon_1`.`calitp_url_number`, 
  `anon_1`.`shape_id`, 
  `anon_2`.`calitp_deleted_at`, 
  `anon_2`.`pt_array`, 
  `anon_2`.`calitp_extracted_at` 
FROM 
  (
    SELECT 
      `anon_3`.`calitp_itp_id` AS `calitp_itp_id`, 
      `anon_3`.`calitp_url_number` AS `calitp_url_number`, 
      `anon_3`.`shape_id` AS `shape_id` 
    FROM 
      (
        SELECT 
          `anon_4`.`calitp_itp_id` AS `calitp_itp_id`, 
          `anon_4`.`calitp_url_number` AS `calitp_url_number`, 
          `anon_4`.`shape_id` AS `shape_id`, 
          `anon_4`.`shape_pt_lat` AS `shape_pt_lat`, 
          `anon_4`.`shape_pt_lon` AS `shape_pt_lon`, 
          `anon_4`.`shape_pt_sequence` AS `shape_pt_sequence`, 
          `anon_4`.`shape_dist_traveled` AS `shape_dist_traveled`, 
          `anon_4`.`calitp_extracted_at` AS `calitp_extracted_at`, 
          `anon_4`.`calitp_hash` AS `calitp_hash`, 
          `anon_4`.`shape_key` AS `shape_key`, 
          `anon_4`.`calitp_deleted_at` AS `calitp_deleted_at` 
        FROM 
          (
            SELECT 
              `views.gtfs_schedule_dim_shapes`.`calitp_itp_id` AS `calitp_itp_id`, 
              `views.gtfs_schedule_dim_shapes`.`calitp_url_number` AS `calitp_url_number`, 
              `views.gtfs_schedule_dim_shapes`.`shape_id` AS `shape_id`, 
              `views.gtfs_schedule_dim_shapes`.`shape_pt_lat` AS `shape_pt_lat`, 
              `views.gtfs_schedule_dim_shapes`.`shape_pt_lon` AS `shape_pt_lon`, 
              `views.gtfs_schedule_dim_shapes`.`shape_pt_sequence` AS `shape_pt_sequence`, 
              `views.gtfs_schedule_dim_shapes`.`shape_dist_traveled` AS `shape_dist_traveled`, 
              `views.gtfs_schedule_dim_shapes`.`calitp_extracted_at` AS `calitp_extracted_at`, 
              `views.gtfs_schedule_dim_shapes`.`calitp_hash` AS `calitp_hash`, 
              `views.gtfs_schedule_dim_shapes`.`shape_key` AS `shape_key`, 
              `views.gtfs_schedule_dim_shapes`.`calitp_deleted_at` AS `calitp_deleted_at` 
            FROM 
              `views.gtfs_schedule_dim_shapes`
          ) AS `anon_4` 
        WHERE 
          `anon_4`.`calitp_extracted_at` <= '2022-1-1' 
          AND `anon_4`.`calitp_deleted_at` > '2022-1-1'
      ) AS `anon_3` 
    WHERE 
    {or_condition}
  ) AS `anon_1` 
  JOIN (
    SELECT 
      `views.gtfs_schedule_dim_shapes_geo`.`calitp_itp_id` AS `calitp_itp_id`, 
      `views.gtfs_schedule_dim_shapes_geo`.`calitp_url_number` AS `calitp_url_number`, 
      `views.gtfs_schedule_dim_shapes_geo`.`shape_id` AS `shape_id`, 
      `views.gtfs_schedule_dim_shapes_geo`.`calitp_extracted_at` AS `calitp_extracted_at`, 
      `views.gtfs_schedule_dim_shapes_geo`.`calitp_deleted_at` AS `calitp_deleted_at`, 
      `views.gtfs_schedule_dim_shapes_geo`.`pt_array` AS `pt_array` 
    FROM 
      `views.gtfs_schedule_dim_shapes_geo`
  ) AS `anon_2` ON `anon_1`.`calitp_itp_id` = `anon_2`.`calitp_itp_id` 
  AND `anon_1`.`calitp_url_number` = `anon_2`.`calitp_url_number` 
  AND `anon_1`.`shape_id` = `anon_2`.`shape_id`
'''
)
"""

In [4]:
def unpack_list_make_or_statement(INCLUDE_ITP_LIST):
    new_cond = ""

    for i in range(0, len(INCLUDE_ITP_LIST)):
        cond = f"calitp_itp_id = {INCLUDE_ITP_LIST[i]}"
        if i==0:
            new_cond = "(" + cond
        elif (i > 0) and (i != len(INCLUDE_ITP_LIST)-1):
            new_cond = new_cond + " OR " + cond
        elif i == len(INCLUDE_ITP_LIST) - 1:
            new_cond = new_cond + " OR " + cond + ")"
    return new_cond

In [5]:
INCLUDE_ME = [17, 4, 100]
itp_id_or_statement = unpack_list_make_or_statement(INCLUDE_ME)
itp_id_or_statement

'(calitp_itp_id = 17 OR calitp_itp_id = 4 OR calitp_itp_id = 100)'

In [6]:
SELECTED_DATE = '2022-1-1'
date_condition = f'(calitp_extracted_at <= {SELECTED_DATE} AND calitp_deleted_at > {SELECTED_DATE})'

In [7]:
condition = itp_id_or_statement + " AND " + date_condition
condition

'(calitp_itp_id = 17 OR calitp_itp_id = 4 OR calitp_itp_id = 100) AND (calitp_extracted_at <= 2022-1-1 AND calitp_deleted_at > 2022-1-1)'

In [8]:
sql_statement = (
f'''
SELECT
    calitp_itp_id,               
    calitp_url_number,
    shape_id,
    pt_array

FROM `views.gtfs_schedule_dim_shapes`

INNER JOIN `views.gtfs_schedule_dim_shapes_geo` 
    USING (calitp_itp_id, calitp_url_number, shape_id)

WHERE {condition}
LIMIT 10
'''
)

In [9]:
query_sql(sql_statement)

DatabaseError: (google.cloud.bigquery.dbapi.exceptions.DatabaseError) 400 Column name calitp_extracted_at is ambiguous at [13:77]

Location: us-west2
Job ID: e1befb49-0cf9-4662-ab73-1f35557c139b

[SQL: 
SELECT
    calitp_itp_id,               
    calitp_url_number,
    shape_id,
    pt_array

FROM `views.gtfs_schedule_dim_shapes`

INNER JOIN `views.gtfs_schedule_dim_shapes_geo` 
    USING (calitp_itp_id, calitp_url_number, shape_id)

WHERE (calitp_itp_id = 17 OR calitp_itp_id = 4 OR calitp_itp_id = 100) AND (calitp_extracted_at <= 2022-1-1 AND calitp_deleted_at > 2022-1-1)
LIMIT 10]
(Background on this error at: https://sqlalche.me/e/14/4xp6)

## With datetime as date

In [ ]:
SELECTED_DATE = datetime.date(2022, 1, 1)

start = datetime.datetime.now()

df = (tbl.views.gtfs_schedule_dim_shapes()
      >> filter(_.calitp_extracted_at <= SELECTED_DATE, 
                _.calitp_deleted_at > SELECTED_DATE
               )
      >> filter(_.calitp_itp_id == 182)
      >> select(_.calitp_itp_id, _.calitp_url_number, _.shape_id)
      >> inner_join(_, 
                    tbl.views.gtfs_schedule_dim_shapes_geo(),
                    ["calitp_itp_id", "calitp_url_number", "shape_id"])
     )
end = datetime.datetime.now()
print(end - start)

## Handle subsets of operators 

* By default, exclude `ITP_ID==200`, but what if we just want 5 operators?
* Instead of `collect` + `isin`, can we construct `or` statement?

In [ ]:
def unpack_list_and_use_or_statement(col, my_list):
    new_or_statement = ""
    
    for i in range(0, len(my_list)):
        add_me = f"(_[{col}]=={my_list[i]})"
        
        if i==0:
            new_or_statement = add_me
        else:
            new_or_statement = new_or_statement + " or " + add_me

    return new_or_statement

In [ ]:
INCLUDE_ITP_ID = [17, 100]

sentence = unpack_list_and_use_or_statement("calitp_itp_id", INCLUDE_ITP_ID)
sentence

In [ ]:
from inspect import cleandoc

cleandoc(sentence)

In [ ]:
# Without collect(), which takes awhile

start = datetime.datetime.now()

df = (tbl.views.gtfs_schedule_dim_shapes()
      >> filter(_.calitp_extracted_at <= SELECTED_DATE, 
                _.calitp_deleted_at > SELECTED_DATE
               )
      >> filter(cleandoc(
          unpack_list_and_use_or_statement(
              "calitp_itp_id", INCLUDE_ITP_ID
          )
      ))
      >> select(_.calitp_itp_id, _.calitp_url_number, _.shape_id)
      >> inner_join(_, 
                    tbl.views.gtfs_schedule_dim_shapes_geo(),
                    ["calitp_itp_id", "calitp_url_number", "shape_id"])
     )


end = datetime.datetime.now()
print(end - start) 

In [ ]:
start = datetime.datetime.now()

df = (tbl.views.gtfs_schedule_dim_shapes()
      >> filter(_.calitp_extracted_at <= SELECTED_DATE, 
                _.calitp_deleted_at > SELECTED_DATE
               )
      >> filter(_.calitp_itp_id==17)
      >> select(_.calitp_itp_id, _.calitp_url_number, _.shape_id)
      >> inner_join(_, 
                    tbl.views.gtfs_schedule_dim_shapes_geo(),
                    ["calitp_itp_id", "calitp_url_number", "shape_id"])
     )

df2 = (tbl.views.gtfs_schedule_dim_shapes()
      >> filter(_.calitp_extracted_at <= SELECTED_DATE, 
                _.calitp_deleted_at > SELECTED_DATE
               )
      >> filter(_.calitp_itp_id==4)
      >> select(_.calitp_itp_id, _.calitp_url_number, _.shape_id)
      >> inner_join(_, 
                    tbl.views.gtfs_schedule_dim_shapes_geo(),
                    ["calitp_itp_id", "calitp_url_number", "shape_id"])
     )


end = datetime.datetime.now()
print(end - start)      

In [ ]:
query_sql('''
df 
UNION
df2
'''
)

In [ ]:
start = datetime.datetime.now()

df = (tbl.views.gtfs_schedule_dim_shapes()
      >> filter(_.calitp_extracted_at <= SELECTED_DATE, 
                _.calitp_deleted_at > SELECTED_DATE
               )
      >> select(_.calitp_itp_id, _.calitp_url_number, _.shape_id)
      >> collect()
      >> filter(_.calitp_itp_id.isin(INCLUDE_ITP_ID))
     )

end = datetime.datetime.now()
print(end - start)  

In [ ]:
(tbl.views.gtfs_schedule_dim_shapes() 
 >> filter(_.calitp_itp_id.isin([100, 17])
          )
)

In [ ]:
start = datetime.datetime.now()

def make_linestring(x):

    # shapely errors if the array contains only one point
    if len(x) > 1:
        # each point in the array is wkt
        # so convert them to shapely points via list comprehension
        as_wkt = [shapely.wkt.loads(i) for i in x]
        return shapely.geometry.LineString(as_wkt)

# apply the function
df['geometry'] = df.pt_array.apply(make_linestring)

# convert to geopandas; geometry column contains the linestring
gdf = gpd.GeoDataFrame(df, geometry = 'geometry', crs=WGS84)

end = datetime.datetime.now()
print(end - start)